WMSDF Code Lab Icon
===================

此程序近似绘制了 WMSDF Code Lab 的图标。

该图标中含有渐变色，这不是 Python turtle 的内置功能，而如果逐像素处理，速度会非常缓慢。

可以考虑通过绘制直线来绘制含有线性渐变色的多边形，而这些直线可以利用解析几何方法求出。

此程序通过类定义了点（二维向量）、直线和半平面，并通过它们实现了近似绘制渐变多边形的工具，最后利用工具绘制图形。其中利用了解析几何和向量的知识。

<span class="hover-show">Python 可能不是慢如乌龟，但 turtle 真的是（也河里，本来就是海龟嘛</span>


In [1]:
import turtle as tt
from math import *
from timeit import default_timer as timer
eps = 1e-7

In [2]:
# 创建窗口
tt.screensize(360,360)
tt.title('WMSDF Code Lab Icon')
tt.speed(0)
tt.delay(0)
tt.tracer(False)

In [3]:
# 初始化
tt.st()
tt.pu()

In [4]:
# 点
'点或二维向量'
class Point:
	x = 0
	y = 0
	def __init__(self,x=0,y=0):
		self.x = x
		self.y = y
	def __repr__(self):
		return '<Point (%.2lf, %.2lf)>' % (self.x, self.y)
	def __add__(self,other):
		if(isinstance(other,Point)):
			return Point(self.x + other.x, self.y + other.y)
		return NotImplemented
	def __sub__(self,other):
		if(isinstance(other,Point)):
			return Point(self.x - other.x, self.y - other.y)
		return NotImplemented
	def __pos__(self):
		return self
	def __neg__(self):
		return Point(0,0) - self
	'数乘或点积'
	def __mul__(self,other):
		if(isinstance(other,(int,float))):
			return Point(self.x * other, self.y * other)
		if(isinstance(other,Point)):
			return self.x * other.x + self.y * other.y
		return NotImplemented
	def __rmul__(self,other):
		return self * other
	def __truediv__(self,other):
		if(isinstance(other,(int,float))):
			return self * (1 / other)
		return NotImplemented
	'叉积的值'
	def __matmul__(self,other):
		if(isinstance(other,Point)):
			return self.x * other.y - self.y * other.x
		return NotImplemented
	'旋转（返回式）'
	def rotate(self,theta):
		return Point(
			self.x * cos(theta) - self.y * sin(theta),
			self.x * sin(theta) + self.y * cos(theta))
	'长度'
	def len(self):
		return (self.x ** 2 + self.y ** 2) ** 0.5
	'同向单位向量'
	def unit(self):
		len = self.len()
		if(abs(len) < eps):
			return Point(0,0)
		return self / len
	'方向角 [0,2π)'
	def theta(self):
		u = self.unit()
		c = u.x
		s = u.y
		if(abs(s) < eps):
			if(c > 0):
				return 0
			return pi
		if(s > 0):
			return acos(c)
		return 2 * pi - acos(c)
		
	'转换为 Tuple'
	def tuple(self):
		return (self.x,self.y)

In [5]:
# 直线一般式
'直线'
class Line:
	A = 0
	B = 0
	C = 0
	'一般式初始化'
	def __init__(self,A=0,B=0,C=0):
		if(isinstance(A,Line)):
			self.A, self.B, self.C = A.A, A.B, A.C  # 是友军，直接初始化
			self.normalize()
			return
		self.A, self.B, self.C = A, B, C
		self.normalize()
	'重置为无效值'
	def clear(self):
		self.A = self.B = self.C = 0
	'输出'
	def __repr__(self):
		if(not self.is_valid()):
			return '<Line invalid>'  # 无效直线
		return ('<Line (%.2f)x + (%.2f)y + (%.2f) = 0>') % (self.A,self.B,self.C)
	'直线是否有效'
	def is_valid(self):
		return (self.A != 0) or (self.B != 0)
	'求倾斜角 (-π/2,π/2]'
	def tilt_angle(self):
		if(not self.is_valid()):
			return nan
		# 为了提高精确度，|A| > |B| 时，使用反斜截式模型
		if(abs(self.B) > abs(self.A)):
			k = -self.A / self.B
			if(k == 0.0):
				k = 0.0
			return atan(k)
		else:
			m = -self.B / self.A
			theta =  pi / 2 - atan(m)
			if(theta > pi / 2):
				theta -= pi
			return theta
	'求斜率'
	def slope(self):
		if(self.B == 0):
			return nan  # 斜率不存在
		return -self.A/self.B
	'一般化（令 A,B 中较小者归一）'  # 经过一般化的直线，可以直接通过比对 A,B,C 的值检验是否相等
	def normalize(self):
		p = min(abs(self.A),abs(self.B))
		if(abs(p) < eps):
			p = max(abs(self.A),abs(self.B))
		if(abs(p) < eps):
			self.C = 0
			return
		if(self.A <= -eps or (abs(self.A) < eps and self.B < 0)):
			p = -p
		self.A /= p
		self.B /= p
		self.C /= p
	'两点式初始化'
	def init_two_points(self,a:Point, b:Point):
		# (a.y - b.y) / (a.x - b.x) = k = -A / B
		self.A = (a.y - b.y)
		self.B = -(a.x - b.x)
		self.C = 0 - (self.A * a.x + self.B * a.y)
		self.normalize()
		return self
	'斜截式初始化'
	def init_kx_b(self,k,b):
		# k = -A / B, (0,b)
		self.A = -k
		self.B = 1
		self.C = -b
		self.normalize()
		return self
	'倾斜角初始化'
	def init_point_theta(self,a:Point,theta):
		# tan(theta) = k = -A / B
		# A = -sin(theta)
		# B = cos(theta)
		self.A = -sin(theta)
		self.B = cos(theta)
		self.C = 0 - (self.A * a.x + self.B * a.y)
		self.normalize()
		return self
	'点是否属于直线'
	def test(self,a:Point):
		return abs(self.A * a.x + self.B * a.y + self.C) < eps
	'求解“特征点”'
	'“特征点”是直线上的一个点，其坐标与 A,B,C 的关系式不分段'
	'是直线 Ax + By + C = 0 和 Bx - Ay = 0 的交点'
	def feature_point(self):
		if(not self.is_valid()):
			return False
		l2 = self.A ** 2 + self.B ** 2
		return Point(- (self.A * self.C) / l2, -(self.B * self.C) / l2)
	'求方向向量'
	def direction_vector(self):
		l = (self.A ** 2 + self.B ** 2) ** 0.5
		dx = -self.B / l
		dy = self.A / l
		return Point(dx,dy)
	'求参数方程点'
	def get_point(self,t):
		d = self.direction_vector()
		p = self.feature_point()
		return Point(d.x * t + p.x, d.y * t + p.y)
	'求点对应的参数'
	def get_time(self,a:Point):
		d = self.direction_vector()
		p = self.feature_point()
		# 此处可以将向量 PA 与单位方向向量作点积
		return d.x * (a.x - p.x) + d.y * (a.y - p.y)
	'在 turtle 上绘制区间（调试用）'
	def draw_seq(self,a,b):
		tt.pu()
		tt.goto(self.get_point(a).tuple())
		tt.pd()
		tt.goto(self.get_point(b).tuple())
		tt.pu()

In [6]:
# 半平面
'半平面'  # 半平面不等号始终使用大于等于号，不另行存储
class HalfPlane(Line):  # 半平面蕴含比直线更多的信息，且其边界直线是直线，故继承直线，使其能当作直线使用
	'一般式初始化（大于等于号）'
	def __init__(self,A=0,B=0,C=0):
		super(HalfPlane,self).__init__(A,B,C)
	'输出'
	def __repr__(self):
		if(not self.is_valid()):
			return '<HalfPlane invalid>'
		return ('<HalfPlane (%.2f)x + (%.2f)y + (%.2f) %s 0>') % (self.A,self.B,self.C,'>=')
	'半平面是否有效'
	def is_valid(self):
		return (self.A != 0) or (self.B != 0)
	'一般化（令 A,B 中较小者归一）'  # 与直线不同。这里不改变各项的符号，故不会改变不等号
	def normalize(self):
		p = min(abs(self.A),abs(self.B))
		if(abs(p) < eps):
			p = max(abs(self.A),abs(self.B))
		if(abs(p) < eps):
			self.C = 0
			return
		self.A /= p
		self.B /= p
		self.C /= p
	'斜截式初始化'
	def init_kx_b(self,k,b):  # 使用大于等于号
		return super(HalfPlane, self).init_kx_b(k,b)
	'两点式 初始化'  # 半平面区域在向量 AB 的左侧
	def init_two_points(self, a:Point, b:Point):
		return super(HalfPlane, self).init_two_points(a,b)
	'倾斜角 初始化'  # 半平面区域在正方向的左侧
	def init_point_theta(self, a:Point, theta):
		return super(HalfPlane, self).init_point_theta(a,theta)
	'点是否属于半平面'
	def test(self, a:Point):
		return self.A * a.x + self.B * a.y + self.C > -eps

In [7]:
# 使用种子的简单随机化工具
# 注：绘图中用了随机化，这是为了保证每次作出的图形完全一致

'使用种子的简单随机化工具（C++ 的 rand 实现）'
class SeededRandom:
	status = 0
	def __init__(self,seed:int=0):
		self.status = seed
	'设置种子'
	def srand(self,seed:int):
		self.status = seed
	'生成随机数'
	def rand(self):
		self.status = 214013 * self.status + 2531011
		self.status = self.status & ((1 << 32) - 1)  # Python 不会自然溢出
		return (self.status >> 16) & ((1 << 15) - 1)

In [8]:
# 求解直线的交点

'求解两直线交点'
def line_intersection(l1:Line, l2:Line):
	# x = (B1C2 - C1B2) / (A1B2 - B1A2)
	# y = (C1A2 - A1C2) / (A1B2 - B1A2)
	xc = l1.B * l2.C - l1.C * l2.B
	yc = l1.C * l2.A - l1.A * l2.C
	p  = l1.A * l2.B - l1.B * l2.A
	if(abs(p) < eps):  # 分母为 0
		if(min(abs(xc),abs(yc)) >= eps):  # 分子不为 0，无解
			return False
		else:  # 分子为 0，重合
			return True
	return Point(xc / p, yc / p)  # 可以求解

In [9]:
# 凸多边形的形态处理

'求三点构成三角形的有向面积'
'参数顺序：基准点、向量起点、向量终点'
def directed_triangle_area(base:Point,start:Point,end:Point):
	vec1 = start - base
	vec2 = end - start
	return (vec1 @ vec2) / 2

'任意多边形的有向面积'
'此方法类似于计算磁通量面积的一般方法。若面积为正，则为逆时针，否则不是'
def directed_area(_points:list[Point]):
	if(len(_points) == 0):
		return 0
	base = Point(0,0)
	points = list(_points)
	points.append(points[0])
	area = 0
	# 依次遍历每一个向量
	for i in range(len(points)-1):
		start = points[i]
		end = points[i+1]
		area += directed_triangle_area(base,start,end)
	return area

'多边形转换为逆时针表述'
def polygon_normalize(_points:list[Point]):
	points = list(_points)
	area = directed_area(points)
	if(area < 0):
		points.reverse()
	return points

'多边形扩张或收缩'
def polygon_shift(val,_points:list[Point]):
	points = polygon_normalize(_points)
	if(len(points) < 3):
		return
	# 求出对应的扩张向量
	lst:list[Point] = []
	for i in range(len(points)):
		# 上一个和下一个点，利用 Python 负索引，不需要 if 判断
		prv = points[i-1]
		nxt = points[i-len(points)+1]
		cur = points[i]
		vec1 = cur - prv
		vec2 = nxt - cur
		if(min(vec1.len(),vec2.len()) < eps):
			# 两点重合，则为不合法情况。令此处扩张向量为 0
			lst.append(Point(0,0))
		else:
			# 两向量的方向相加，得到外角平分线方向，右转90°得到指向多边形外的内角平分向量
			vecn:Point = (vec1.unit() + vec2.unit()).rotate(-pi/2)
			if(vecn.len() < eps):
				# 两向量反向共线，则此处为 0° 内角，扩张向量沿 vec1 方向
				vecn = vec1.unit()
			else:
				vecn = vecn.unit()
			cos_theta = (vec1 * vec2) / (vec1.len() * vec2.len())  # 这是外角的余弦
			vecn *= (2 / (1 + cos_theta)) ** 0.5  # 这是内角一半的正弦值的倒数，为扩张向量的模
			lst.append(vecn)
	# 执行扩张
	for i in range(len(points)):
		points[i] += val * lst[i]
	return points

'绘制多边形（调试用）'
def polygon_draw(points:list[Point]):
	if(len(points) == 0):
		return
	tt.pu()
	tt.goto(points[0].tuple())
	tt.pd()
	for i in range(1,len(points)):
		tt.goto(points[i].tuple())
	tt.goto(points[0].tuple())
	tt.pu()

'点是否在多边形内部 - 转角法'
'0=在外部，-1=在边界上，1=在内部'
def polygon_has(a:Point,points:list[Point]):
	# 如果该点就是顶点，则在多边形边界上
	for p in points:
		if((p - a).len() < eps):
			return -1
	theta = (points[-1] - a).theta()
	sum_theta = 0
	# 遍历所有点
	for p in points:
		new_theta = (p - a).theta()
		dtheta = (new_theta - theta) % (2 * pi)
		theta = new_theta
		if(abs(dtheta - pi) < eps):
			# 出现 180° 转弯，说明点在一条边上
			return -1
		if(dtheta > pi):
			dtheta -= 2 * pi
		sum_theta += dtheta
	# 最终旋转角接近 0，则点在多边形外
	if(abs(sum_theta) < eps):
		return 0
	# 如果旋转角接近 360°，则在多边形内
	return 1

'分解为三角形'
'该算法最坏的时间复杂度为 n³'
def polygon_to_triangle(_points:list[Point]):
	points = polygon_normalize(_points)
	if(len(points) < 4):
		return [points]
	ret = []
	# 迭代切点，直到剩下一个三角形
	while len(points) > 3:
		cut_point = -1
		for i in range(len(points)):
			p0 = points[i-1]
			p2 = points[i-len(points)+1]
			p1 = points[i]
			# 条件一：中间点是凸点
			if((p1 - p0) @ (p2 - p1) < 0):
				continue
			is_ok = True
			# 条件二：不存在一个点在三角形内
			for p in points:
				if(polygon_has(p,[p0,p1,p2]) > 0):
					is_ok = False
					break
			# 标记为切点，退出循环
			if(is_ok):
				ret.append([p0,p1,p2])  # 存储答案
				cut_point = i
				break
		if(cut_point == -1):
			# 未能删除点，有问题
			raise ValueError('Cannot remove all ears!',str(points))
		points[cut_point:cut_point+1] = []  # 删除切点
	# 最终的三角形加入答案
	ret.append(points)
	return ret

In [10]:
# 凸多边形、半平面交与直线截得的线段

'凸多边形点转半平面交'
def convex_to_halfplane(_points:list[Point]):
	points = polygon_normalize(_points)
	ret = []
	for i in range(len(points)):
		ret.append(HalfPlane().init_two_points(points[i-1],points[i]))
	return ret

'单个半平面截取直线'
def halfplane_cut(l:Line,h:HalfPlane):
	p = line_intersection(l,h)
	if(isinstance(p,bool)):  # 交点不唯一
		if(h.test(l.get_point(0))):  # 无约束
			return [+inf,-1]
		else:  # 无取值
			return [-inf,-1]
	t = l.get_time(p)
	# 通过测试 t - 1 判断不等号
	if(h.test(l.get_point(t-1))):
		return [t,-1]  # t_x <= t
	return [t,+1]  # t_x >= t

'半平面交截取直线'
def halfplane_int_cut(l:Line,lines:list[HalfPlane]):
	min_val = -inf
	max_val = inf
	for h in lines:
		c = halfplane_cut(l,h)
		if(c[1] > 0):  # 大于等于号指定最小值
			min_val = max(min_val,c[0])
		else:
			max_val = min(max_val,c[0])
	return [min_val,max_val]

'近似绘制渐变多边形！'
'@param points      多边形'
'@param base        零点'
'@param theta       倾斜角'
'@param color_func  颜色函数，自变量为直线距离零点的距离，以向右，斜向右或竖直向上为正'
'@param width       线宽'
'@param rg          线距区间'
def draw_pan_polygon(points:list[Point],base:Point,theta,color_func,width,rg:list,line_only=False,scatter=1.3,scatter_seed=19260817):
	rd = SeededRandom(scatter_seed)
	tt.pu()
	# 多边形按画笔半径收缩
	points = polygon_shift(-width/2, points)
	narrow_points = polygon_shift(-width/8-scatter, points)
	tt.color('black')
	# 转化为一些三角形
	triangles = polygon_to_triangle(points)
	narrow_triangles = polygon_to_triangle(narrow_points)
	tt.width(1)
	for tr in (triangles if line_only else narrow_triangles):
		polygon_draw(tr)
	tt.width(width)
	# 将三角形变成半平面交
	planes = []
	for tr in triangles:
		planes.append(convex_to_halfplane(tr))
	if(line_only): return
	# 构造初始线
	l0 = Line().init_point_theta(base,theta)
	# 对于每个线
	for delta in rg:
		# 构造线
		l1 = Line(l0)
		l1.C += delta * (l1.A ** 2 + l1.B ** 2) ** 0.5
		# 颜色函数
		tt.color(color_func(-delta))
		# 对于三角形
		rng_lst = []
		for tr in planes:
			l_rng = halfplane_int_cut(l1,tr)
			if(l_rng[0] <= l_rng[1]): rng_lst.append(l_rng)
		# 排序列表
		rng_lst.sort()
		lt = 0
		rt = 0
		while lt < len(rng_lst):
			rt = lt
			lf = rng_lst[lt][0]
			rf = rng_lst[rt][1]
			# 首尾相连的线段合并为一条绘制
			while(rt < len(rng_lst) - 1 and rng_lst[rt+1][0] - rng_lst[rt][1] < eps):
				rt += 1
				rf = rng_lst[rt][1]
			# 抖动边界，使边界变得不规则，从而消除失真影像 ——NVIDIA
			lf += scatter * 2 * (0.5 - ((rd.rand() & 2047) / 2048))
			rf += scatter * 2 * (0.5 - ((rd.rand() & 2047) / 2048))
			l1.draw_seq(lf,rf)
			lt = rt + 1

In [11]:
# 颜色处理

'颜色线性混合'
def color_blend(a:tuple,b:tuple,c):
	d = 1 - c
	return (
		a[0] * d + b[0] * c,
		a[1] * d + b[1] * c,
		a[2] * d + b[2] * c
	)

'颜色函数混合'
def color_blend_func(a:tuple,b:tuple,left,right,t):
	r = (t - left) / (right - left)
	r = min(1,max(0,r))
	return color_blend(a,b,r)

'实数区间'
def float_range(a,b,c):
	if(c == 0):
		return []
	lst = []
	i = a
	while i * c <= b * c:
		lst.append(i)
		i += c
	return lst

'颜色转换'
def color_dac(a):
	return [a[0] / 255, a[1] / 255, a[2] / 255]

In [12]:
# 绘图

d = 1.7  # 等色线线距
w = 4  # 宽度
line_only = False

tic = timer()

# 绘制外层正方形
psquare = [Point(240,0),Point(0,240),Point(-240,0),Point(0,-240)]
draw_pan_polygon(
	points = psquare,
	base = Point(0,0),
	theta = Line(0.725,0.275,0).tilt_angle(),
	color_func = lambda x: color_dac(color_blend_func((232, 234, 246), (243, 229, 245),-240,240,x)),
	width = w,
	rg = float_range(-240,240,d),
	line_only = line_only
)
tt.width(10)
tt.pencolor(color_dac((219, 214, 152)))
# polygon_draw(psquare)

# 中心文
sz = 440
tt.color(color_dac((121, 134, 203)))
tt.goto((-sz * 0.02,-sz * 2.515))
tt.write("\uE113", font=('EOP Score',sz,'bold'), align='center')

# 生成左侧尖括号
ratio = 0.5
bracket_inset = 50
bracket_outset = 30
bracket_size = 240 * ratio
p1:list[Point] = [Point() for i in range(6)]
p1[0] = Point(-240 + bracket_inset, 0)
p1[3] = Point(-240 - bracket_outset, 0)
p1[1] = Point(-240, 0) + bracket_size * Point(1,1) + bracket_inset * Point(0.5,-0.5)
p1[2] = Point(-240, 0) + bracket_size * Point(1,1) - bracket_outset* Point(0.5,-0.5)
p1[4] = Point(p1[2].x, -p1[2].y)
p1[5] = Point(p1[1].x, -p1[1].y)

upper = p1[2].y
lower = p1[4].y

# 绘制左侧尖括号
draw_pan_polygon(
	points = p1,
	base = Point(0,0),
	theta = 0,
	color_func = lambda x: color_dac(color_blend_func((171, 71, 188), (206, 147, 216),lower/3,upper/3,x)),
	width = w,
	rg = float_range(-240,240,d),
	line_only = line_only
)
tt.width(12)
tt.pencolor(color_dac((206, 147, 216)))
# polygon_draw(p1)

# 翻转尖括号图案
p2:list[Point] = []
for p in p1:
	p2.append(Point(-p.x,p.y))

# 绘制右侧尖括号
draw_pan_polygon(
	points = p2,
	base = Point(0,0),
	theta = 0,
	color_func = lambda x: color_dac(color_blend_func((41, 182, 246), (129, 212, 250),lower/3,upper/3,x)),
	width = w,
	rg = float_range(-240,240,d),
	line_only = line_only
)
tt.width(12)
tt.pencolor(color_dac((129, 212, 250)))
# polygon_draw(p2)

# 生成斜杠
slash_width = 67
slash_height = 532
slash_theta = -21 / 180 * pi
p3 = [
	Point(+slash_width/2,+slash_height/2),
	Point(-slash_width/2,+slash_height/2),
	Point(-slash_width/2,-slash_height/2),
	Point(+slash_width/2,-slash_height/2)
]

for i in range(len(p3)):  # 旋转
	p3[i] = p3[i].rotate(slash_theta)

# 绘制斜杠
# draw_pan_polygon(
# 	points = p3,
# 	base = Point(0,0),
# 	theta = - pi / 4,
# 	color_func = lambda x: color_dac(color_blend_func((210,190,80),(244,151,12),p3[3].y / 1.41,p3[1].y / 1.41,x)),
# 	width = w,
# 	rg = float_range(-400,400,d),
# 	line_only = line_only
# )

# 边线
border_sz = 271
tt.width(1)
tt.color((0,0,0))
pborder = [Point(border_sz,border_sz),Point(-border_sz,border_sz),Point(-border_sz,-border_sz),Point(border_sz,-border_sz)]
polygon_draw(pborder)

# 计时
toc = timer()
print('Used Time:', str(toc-tic) + 's')

Used Time: 0.1009407999999894s


In [13]:
# 完事
tt.pu()
tt.ht()
tt.done()
tt.update()

Terminator: 